# Trabalho Prático 2: Realidade Aumentada

## Alunos: 

- Felipe Eduardo dos Santos - 2017021223
- Renan Antunes Braga Bomtempo - 2018048524

## Dependências


In [2]:
# OpenCV
import cv2

import numpy as np

# OpenGL
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from objloader import *

## Funções Auxiliares (OpenGL)

In [3]:
'''
Inicialização dos parâmetros da OpenGL.
'''
def initOpenGL(focal_len, dimensions):
    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
 
    fx = focal_len[0]
	fy = focal_len[1]
	fovy = 2*np.arctan(0.5*height/fy)*180/np.pi
	aspect = (width*fy)/(height*fx)
    gluPerspective(fovy, aspect, 0.1, 100.0)


def idleCallback():
    glutPostRedisplay()

## Encontrar o alvo na cena

In [ ]:
def detect_markers():
    return

## Posicionar o modelo em cima do alvo

In [ ]:
'''
Posiciona um objeto 3D em cima do alvo detectado na cena.
'''
def object3D(img_pts, world_pts, camera_matrix, dist_coef, obj):
    # Calcular matrix de projeção
    _, rot_vecs, t_vecs = cv2.solvePnP(world_pts, img_pts, camera_matrix, dist_coef)
    rot_m = cv2.Rodrigues(rot_vecs)[0]

	proj_matrix = np.array([[rot_m[0][0], rot_m[0][1], rot_m[0][2], t_vecs[0]], 
		                    [rot_m[1][0], rot_m[1][1], rot_m[1][2], t_vecs[1]], 
		                    [rot_m[2][0], rot_m[2][1], rot_m[2][2], t_vecs[2]],
		                    [        0.0,         0.0,         0.0,      1.0]])

	# Mudança de sistema de coordenadas (OpenCV -> OpenGL)
	flip_yz = np.array([[1, 0,  0, 0],
		                [0, 1,  0, 0],
		                [0, 0, -1, 0],
		                [0, 0,  0, 1]])

	proj_matrix = np.dot(flip_yz, proj_matrix)

	glLoadMatrixd(np.transpose(proj_matrix))

    # Renderiza o modelo do Pikachu
    glCallList(obj.gl_list)

In [ ]:
'''
Loop de renderização
'''
def displayCallback():
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # carregar o modelo 3D do Pikachu
    obj = OBJ("Pikachu.obj", swapyz=True)

    # habilita o uso de texturas (o Pikachu tem textura)
    glEnable(GL_TEXTURE_2D)

    # Encontra os alvos na cena
    alvos = detect_markers()

    for alvo in alvos:
        # Posiciona o modelo 3D em cima do alvo
        object3D(alvo[0], alvo[1], camera_matrix, dist_coef, obj) 
        glutSwapBuffers()     

# Main Function

In [4]:
if __name__ == '__main__':
    # Intrinsic paremeters from camera calibration
    focal_len = (1295.66495, 1280.53452)
    princ_pt  = ( 915.60124,  478.74546)
    dist_coef = np.array([0.06646, 0.27952, -0.00221, -0.00802, 0.])
    
    # Camera matrix
    camera_matrix = np.array([[focal_len[0], 0.          , princ_pt[0]],
                              [0.          , focal_len[1], princ_pt[1]],
                              [0.          , 0.          , 1.        ]])

    # Open input video
    vid = cv2.VideoCapture('tp2-icv-input.mp4')

    # Viewport dimensions
    dimensions = (640, 480)

    # GLUT configuration
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    glutInitWindowSize(*dimensions)
    window = glutCreateWindow(b'Realidade Aumentada')

    # OpenGL intialization
    initOpenGL(focal_len, dimensions)

    glutDisplayFunc(displayCallback)
    glutIdleFunc(idleCallback)
    
    glutMainLoop()

    vid.release()